# Outline of thought process
- starting out with the creature class as the base class. The creature class will be used to create monsters/creatures that<br>
have not become npcs

- the pc_class class will be used to give player characters access to class mechanics
- the pc class will inherit from the creature class and the pc_class class to create a pc
- the npc class will inherit from the creature class and set the is_npc flag of an object to True so that a monster can be <br>
changed to npc status and stored to come back as a recurring character
- the item class will be used to create all items(including interactable objects our pcs might run into or use to change the <br> course of a battle. Think setting a fire to obscure the battle field or wedging a door shut


In [ ]:
### start of a creature class


name = {'name':'aarakocra', 'abilities':{},'hp':13, 'alt_hp':[8, 3], 'ac': 12, 
        'actions':['attack', 'cast', 'dash', 'disengage', 'dodge', 'help', 'hide','ready', 'search', 'use an object', 'improvise'],
        'bonus_actions': {}, 'action_count': 0, 'b_action_count': 0, 'reaction_count': 0, 'feats':{}, 'traits':{}, 'initiative': 2, 
        'inventory':{}, 'vision': None, 'movement':{'walking': 20, 'flight': 50, 'swimming': None,'climbing': None}, 
        'main_hand': None, 'off_hand': None, 'armor': None,'disposition':'indifferent'}
class Creature:
    def __init__(self):
        pass
    
    def create_creature(self, dictionary):
        setattr(self, 'species', dictionary['name'])
        setattr(self, 'abilities', dictionary['abilities'])
        setattr(self, 'hp', dictionary['hp'])
        setattr(self, 'alt_hp', dictionary['alt_hp'])
        setattr(self, 'ac', dictionary['ac'])
        setattr(self, 'actions', dictionary['actions'])
        setattr(self, 'bonus_actions', dictionary['bonus_actions'])
        setattr(self, 'feats', dictionary['feats'])
        setattr(self, 'traits', dictionary['traits'])
        setattr(self, 'initiative_bonus', dictionary['initiative'])
        setattr(self, 'initiative', None)
        setattr(self, 'inventory', dictionary['inventory'])
        setattr(self, 'vision', dictionary['vision'])
        setattr(self, 'movement', dictionary['movement'])
        setattr(self, 'main_hand', dictionary['main_hand'])
        setattr(self, 'off_hand', dictionary['off_hand'])
        setattr(self, 'armor', dictionary['armor'])
        setattr(self, 'disposition', dictionary['disposition'])

    def initiative(self):
        setattr(self, d20() + self.initiative_bonus)
    #def item_equip(self, item):
    #def add_inventory:


In [ ]:
# keeping pc_class and pc as different classes to account for multi-classing

class PcClass:
    pass
    

In [ ]:
class Pc(Creature, PcClass):
    def __init__(self):
        pass
    def create_pc(self):
        setattr(self, 'pc', True)
    

In [ ]:
class Npc(Creature):
    def __init__(self):
        pass
    def create_npc(self):
        setattr(self, 'npc', True)

In [ ]:
class item:
    pass